 1. design model,inputs and outputs size, and forward pass
 2. construct loss and optimizer
 3. training loop
   - forward pass
   - backward pass
   - update weights

In [2]:
import torch
import torch.nn as nn
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt
import time

- with cpu 

In [4]:
# 这里面的data都是叶子节点张量，而requires_grad的都是神经网络里的参数，这些参数会自动赋予grad的
# ------------------------
# 0. prepare data

bc = datasets.load_breast_cancer()   # 乳腺癌 二分类问题，根据输入的特征预测
# print(bc)
x, y = bc.data, bc.target
print(x.shape, y.shape)
print(x.dtype, y.dtype)
print(type(x), type(y))   # 可以看出属于numpy

# x.shape是元祖，以下两种都行，第二种更简便
# n_samlpes, n_features = x.shape[0], x.shape[1]
n_samlpes, n_features = x.shape
print(f'n_samples: {n_samlpes}, n_features: {n_features}')

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

# scaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(type(x_train), type(x_test), type(y_train), type(y_test))

# 将numpy转为tensor
x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))
print(type(x_train), x_train.dtype)
print(type(y_train), y_train.dtype) 

# for i in zip(x_train, x_test, y_train, y_test):
#     print(f'"{i}": type({i}), {i}.dtype , {i}.shape, "\n')

# 将y数据转换为列向量
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


# 1. model
# f = wx + b, sigmoid at the end
class LogisticRegression(nn.Module):
    
    def __init__(self, n_input_features):
        super().__init__()
        self.Linear = nn.Linear(n_input_features, 1)
    
    def forward(self, x):
        # 模型中需要包含激活函数，二分类问题的输出层用sigmoid
        # y_predicted = nn.Sigmoid(self.Linear(x))   # nn和torch下面的sigmoid有什么区别？
        y_predicted = torch.sigmoid(self.Linear(x))
        return y_predicted
    
model = LogisticRegression(n_features)

# 2. loss and optimizer
lr = 0.01
criterion = nn.BCELoss()   # 用于二分类问题，每个样本只有两个类别
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

# 3. training loop
time_begin = time.time()
epoch = 1000
for i in range(epoch):
    # forward pass
    y_hat = model(x_train)
    loss = criterion(y_hat, y_train)

    # backward pass
    loss.backward()
    
    # update
    optimizer.step()
    
    # grad_zero
    optimizer.zero_grad()

    if (i+1) % 100 == 0:   # 是i+1，从0开始，真正的epoch+1
        print(f'epoch: {i+1}: loss: {loss.item():.8f}')

time_end = time.time()
time_ = time_end - time_begin
print(f'Time: {time_}s')

# 用test数据测试上面的training得到的神经网络的权重
# 对于这个例子从技术上讲是不需要的，但一般是加上，因为为了防止后续再对模型进行更新，因为调用了forward，所以张量会存储用于梯度计算
with torch.no_grad():
    y_hat = model(x_test)
    y_hat_ = y_hat.round()   # 输出1还是0，与二分类判断直接联系起来，
    acc = y_hat_.eq(y_test).sum() / float(y_test.shape[0])   #判断上一步的值与y_test的值是否一致，一致就记为1，然后总和除以数据的长度，就是正确率；用float是为了保证足够精确的小数
    print(f'accuracy = {acc:.4f}')








(569, 30) (569,)
float64 int32
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
n_samples: 569, n_features: 30
(455, 30) (114, 30) (455,) (114,)
(455, 30) (114, 30) (455,) (114,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
<class 'torch.Tensor'> torch.float32
<class 'torch.Tensor'> torch.float32
torch.Size([455, 30]) torch.Size([114, 30]) torch.Size([455, 1]) torch.Size([114, 1])
epoch: 100: loss: 0.24781327
epoch: 200: loss: 0.17749397
epoch: 300: loss: 0.14672913
epoch: 400: loss: 0.12885503
epoch: 500: loss: 0.11694510
epoch: 600: loss: 0.10831949
epoch: 700: loss: 0.10171279
epoch: 800: loss: 0.09644578
epoch: 900: loss: 0.09211907
epoch: 1000: loss: 0.08848123
Time: 1.2732248306274414s
accuracy = 0.9298


In [21]:
torch.cuda.is_available()

True

- with cuda

In [24]:
# 设置device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# 不能用下面这行代码，它不能让torch自动的将所有张量和模型放到GPU中
# torch.set_default_device('cuda')
# ------------------------
# 0. prepare data

bc = datasets.load_breast_cancer()   # 乳腺癌 二分类问题，根据输入的特征预测
# print(bc)
x, y = bc.data, bc.target
print(x.shape, y.shape)
print(x.dtype, y.dtype)
print(type(x), type(y))   # 可以看出属于numpy

# x.shape是元祖，以下两种都行，第二种更简便
# n_samlpes, n_features = x.shape[0], x.shape[1]
n_samlpes, n_features = x.shape
print(f'n_samples: {n_samlpes}, n_features: {n_features}')

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

# scaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(type(x_train), type(x_test), type(y_train), type(y_test))

# 将numpy转为tensor
x_train = torch.from_numpy(x_train.astype(np.float32)).to(device)
x_test = torch.from_numpy(x_test.astype(np.float32)).to(device)
y_train = torch.from_numpy(y_train.astype(np.float32)).to(device)
y_test = torch.from_numpy(y_test.astype(np.float32)).to(device)
print(type(x_train), x_train.dtype)
print(type(y_train), y_train.dtype) 

# for i in zip(x_train, x_test, y_train, y_test):
#     print(f'"{i}": type({i}), {i}.dtype , {i}.shape, "\n')

# 将y数据转换为列向量
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


# 1. model
# f = wx + b, sigmoid at the end
class LogisticRegression(nn.Module):
    
    def __init__(self, n_input_features):
        super().__init__()
        self.Linear = nn.Linear(n_input_features, 1)
    
    def forward(self, x):
        # 模型中需要包含激活函数，二分类问题的输出层用sigmoid
        # y_predicted = nn.Sigmoid(self.Linear(x))   # nn和torch下面的sigmoid有什么区别？
        y_predicted = torch.sigmoid(self.Linear(x))
        return y_predicted
    
model = LogisticRegression(n_features).to(device)

# 2. loss and optimizer
lr = 0.01
criterion = nn.BCELoss()   # 用于二分类问题，每个样本只有两个类别
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

# 3. training loop
time_begin = time.time()
epoch = 1000
for i in range(epoch):
    # forward pass
    y_hat = model(x_train)
    loss = criterion(y_hat, y_train)

    # backward pass
    loss.backward()
    
    # update
    optimizer.step()
    
    # grad_zero
    optimizer.zero_grad()

    if (i+1) % 100 == 0:   # 是i+1，从0开始，真正的epoch+1
        print(f'epoch: {i+1}: loss: {loss.item():.8f}')

time_end = time.time()
time_ = time_end - time_begin
print(f'Time: {time_}s')

# 用test数据测试上面的training得到的神经网络的权重
# 不计算梯度，对于这个例子从技术上讲是不需要的，但一般是加上
# 减少计算资源，可以防止后续再对模型进行更新，因为调用了forward，所以张量会存储用于梯度计算
with torch.no_grad():
    y_hat = model(x_test)
    y_hat_ = y_hat.round()
    acc = y_hat_.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')



(569, 30) (569,)
float64 int32
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
n_samples: 569, n_features: 30
(455, 30) (114, 30) (455,) (114,)
(455, 30) (114, 30) (455,) (114,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
<class 'torch.Tensor'> torch.float32
<class 'torch.Tensor'> torch.float32
torch.Size([455, 30]) torch.Size([114, 30]) torch.Size([455, 1]) torch.Size([114, 1])
epoch: 100: loss: 0.24877240
epoch: 200: loss: 0.17892997
epoch: 300: loss: 0.14756942
epoch: 400: loss: 0.12910828
epoch: 500: loss: 0.11677261
epoch: 600: loss: 0.10785934
epoch: 700: loss: 0.10106193
epoch: 800: loss: 0.09566969
epoch: 900: loss: 0.09126195
epoch: 1000: loss: 0.08757325
Time: 2.158806324005127s
accuracy = 0.9474
